In [1]:
using Distributions

A player with a fixed winrate has a number of wins following the [Negative Binomial Distribution](https://en.wikipedia.org/wiki/Negative_binomial_distribution).  However, you can't continue winning indefinitely, so the distribution is truncated with all the probability mass above the maximum number of wins being placed instead on the maximum win amount.

In [2]:
cdfnegbin(k; p, r) = one(k) - cdf(Beta(k+one(k),r),p)
pdfnegbin(k;p,r) = binomial( k+r - one(k), k) * (one(p) - p)^r * (p^k)
meannegbin(p,r) = (p*r) / (1 - p)


function pdfnegbin_truncated(k; p, r, maxwins)
    if k < maxwins
        pdfnegbin(k; p = p, r = r)
    elseif k == maxwins
        pdfnegbin(maxwins; p = p, r = r) + (1 - cdfnegbin(maxwins; p = p, r = r))
    else 
        zero(pdfnegbin(k; p = p, r = r))
    end
end


pdfnegbin_truncated (generic function with 1 method)

In [3]:
@enum Rarity common uncommon rare mythic

Function to compare expected gold for a given winrate as well as gold for duplicates and percentage of playsets of each rarity:

In [4]:
function compare_gold_ev(; winrate, r = 3, maxwins = 7, completion_pct, gold_duplicates, 
        uc_r_upgrade = 0.10, uc_m_upgrade = 0.05, r_m_upgrade = 0.15)
    
    uc_icr = (1 - uc_r_upgrade - uc_m_upgrade) * gold_duplicates[uncommon] * completion_pct[uncommon] +
     uc_r_upgrade * gold_duplicates[rare] * completion_pct[rare] +
     uc_m_upgrade * gold_duplicates[mythic] * completion_pct[mythic]
    
    rare_icr = (1 - r_m_upgrade) * gold_duplicates[rare] * completion_pct[rare] +
     r_m_upgrade * gold_duplicates[mythic] * completion_pct[mythic]
    
    qc_payoff2 = Dict(
        0 => 100 + 3 * uc_icr,
        1 => 200 + 3 * uc_icr,
        2 => 300 + 3 * uc_icr,
        3 => 400 + 3 * uc_icr,
        4 => 500 + 2 * uc_icr + rare_icr,
        5 => 600 + 2 * uc_icr + rare_icr,
        6 => 800 + 1 * uc_icr + 2 * rare_icr,
        7 => 1000 + uc_icr + 2 * rare_icr
    )
    
    qc_payoff1 = Dict(
        0 => 100,
        1 => 200,
        2 => 300,
        3 => 400,
        4 => 500,
        5 => 600,
        6 => 800,
        7 => 1000
    )
    
    x = [ (i, qc_payoff1[i], qc_payoff2[i], pdfnegbin_truncated(i; p = winrate, r = r, maxwins = maxwins)) for i in 0:7]
    
    oldavg = sum( z[2] * z[4] for z in x )
    newavg  = sum( z[3] * z[4] for z in x )
    avgwins = sum( z[1] * z[4] for z in x)
    (oldavg = oldavg, newavg = newavg, ratio = newavg / oldavg, avgwins = avgwins)
end

compare_gold_ev (generic function with 1 method)

Positive expected gold with sub-50% winrate with no rares:

In [5]:
compare_gold_ev(winrate = 0.49;
 completion_pct = Dict(
    common => 1,
    uncommon => 1,
    rare => 0.0,
    mythic => 0.0
), gold_duplicates = Dict(
    common => 25,
    uncommon => 50,
    rare => 100,
    mythic => 200
    )
)

(oldavg = 398.1007976072926, newavg = 506.17869943346034, ratio = 1.2714837610870136, avgwins = 2.7688214308376375)

Another scenario that is more extreme but not too far-fetched for semi-dedicated players long after release:

In [6]:
result = compare_gold_ev(winrate = 0.60;
 completion_pct = Dict(
    common => 1,
    uncommon => 1,
    rare => 0.7,
    mythic => 0.4
), gold_duplicates = Dict(
    common => 25,
    uncommon => 50,
    rare => 100,
    mythic => 200
    )
)

(oldavg = 549.7947136, newavg = 725.7695756800001, ratio = 1.3200737615822724, avgwins = 3.9507655680000004)

In terms of % of pack per win, this is quite a bit higher than Hearthstone, **in addition to** the more frontloaded daily/weekly rewards.  Would it be nice? Sure.  It might be too good to be true though.

In [7]:
(result.newavg - 500) / 1000 / result.avgwins, 10/100/3

(0.05714577891147605, 0.03333333333333333)